In [147]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('stop_words_en.txt') as f:
    stop_words = set(f.read().split())
    
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        if word in stop_words:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)


Overwriting mapper.py


In [148]:
%%writefile counter_process.py
import sys
import re


stop_words = int(re.search('\d+|$', sys.stdin.readline()).group())
all_words = int(re.search('\d+|$', sys.stdin.readline()).group())
print(stop_words/all_words*100)



Overwriting counter_process.py


In [149]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [150]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=0

LOGS="stderr_logs.txt"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming stopwords" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> $LOGS


rm: `wordcount_result_1525928816515795': No such file or directory


In [151]:
%%bash
LOGS="stderr_logs.txt"
cat $LOGS |egrep "*words"| python ./counter_process.py
cat $LOGS >&2

38.44036900909957


18/05/10 05:07:00 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/10 05:07:00 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/05/10 05:07:00 INFO mapred.FileInputFormat: Total input files to process : 1
18/05/10 05:07:00 INFO mapreduce.JobSubmitter: number of splits:2
18/05/10 05:07:00 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1525903343186_0014
18/05/10 05:07:01 INFO impl.YarnClientImpl: Submitted application application_1525903343186_0014
18/05/10 05:07:01 INFO mapreduce.Job: The url to track the job: http://cbccbb36aa92:8088/proxy/application_1525903343186_0014/
18/05/10 05:07:01 INFO mapreduce.Job: Running job: job_1525903343186_0014
18/05/10 05:07:07 INFO mapreduce.Job: Job job_1525903343186_0014 running in uber mode : false
18/05/10 05:07:07 INFO mapreduce.Job:  map 0% reduce 0%
18/05/10 05:07:23 INFO mapreduce.Job:  map 38% reduce 0%
18/05/10 05:07:29 INFO mapreduce.Job:  map 57% reduce 0%
18/05/10 05:07:35 INFO 